# FantasyPros Projections Analysis

This notebook loads the scraped FantasyPros projections and applies custom scoring.

In [1]:
import pandas as pd
import numpy as np
import yaml
import os
from datetime import datetime
import glob

# Display settings
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 150)

In [2]:
# Load the most recent projection files
def load_latest_projections(position='all_positions'):
    """Load the most recent projection file for a position"""
    pattern = f'data/projections/projections_{position}_*.csv'
    files = glob.glob(pattern)
    
    if not files:
        print(f"No projection files found for {position}")
        print("Run 'python scrape_projections.py' first")
        return None
    
    # Get most recent file
    latest_file = sorted(files)[-1]
    print(f"Loading: {latest_file}")
    
    df = pd.read_csv(latest_file)
    return df

# Load all projections
projections = load_latest_projections('all_positions')

if projections is not None:
    print(f"\nLoaded {len(projections)} player projections")
    print(f"Positions: {projections['POSITION'].value_counts().to_dict()}")
    display(projections.head())

Loading: data/projections/projections_all_positions_20250814.csv

Loaded 747 player projections
Positions: {'WR': 231, 'RB': 188, 'TE': 157, 'QB': 102, 'K': 37, 'DST': 32}


,UNNAMED:_0_LEVEL_0_PLAYER,PASSING_ATT,PASSING_CMP,PASSING_YDS,PASSING_TDS,PASSING_INTS,RUSHING_ATT,RUSHING_YDS,RUSHING_TDS,MISC_FL,...,XPT,FPTS,SACK,INT,FR,FF,TD,SAFETY,PA,YDS_AGN
0,Josh Allen BUF,522.4,334.7,3914.7,29.0,10.5,115.7,571.8,10.7,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lamar Jackson BAL,480.4,321.0,3923.2,32.5,8.3,134.7,827.9,3.9,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jalen Hurts PHI,461.5,311.7,3536.8,23.7,8.9,147.2,633.0,12.6,3.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jayden Daniels WAS,536.2,364.4,3899.3,26.9,10.7,136.5,746.8,5.6,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Joe Burrow CIN,626.7,434.1,4635.9,35.9,10.5,48.2,180.1,2.1,3.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Load league configuration
with open('config/league-config.yaml', 'r') as f:
    config = yaml.safe_load(f)

scoring = config['scoring']
print("League Scoring System:")
print(f"Pass TD: {scoring['passing']['touchdown']} pts")
print(f"Rush/Rec TD: {scoring['rushing']['touchdown']} pts")
print(f"PPR: {scoring['receiving'].get('reception', 0)} pts")

League Scoring System:
Pass TD: 4 pts
Rush/Rec TD: 6 pts
PPR: 0 pts


In [4]:
def calculate_fantasy_points(row, scoring_config):
    """Calculate fantasy points based on projections and scoring settings"""
    points = 0
    
    # Passing
    if 'PASSING_YDS' in row and pd.notna(row.get('PASSING_YDS')):
        points += row['PASSING_YDS'] * scoring_config['passing']['yards']
    if 'PASSING_TDS' in row and pd.notna(row.get('PASSING_TDS')):
        points += row['PASSING_TDS'] * scoring_config['passing']['touchdown']
    if 'PASSING_INTS' in row and pd.notna(row.get('PASSING_INTS')):
        points += row['PASSING_INTS'] * scoring_config['passing']['interception']
    
    # Rushing
    if 'RUSHING_YDS' in row and pd.notna(row.get('RUSHING_YDS')):
        points += row['RUSHING_YDS'] * scoring_config['rushing']['yards']
    if 'RUSHING_TDS' in row and pd.notna(row.get('RUSHING_TDS')):
        points += row['RUSHING_TDS'] * scoring_config['rushing']['touchdown']
    
    # Receiving
    if 'RECEIVING_REC' in row and pd.notna(row.get('RECEIVING_REC')):
        points += row['RECEIVING_REC'] * scoring_config['receiving'].get('reception', 0)
    if 'RECEIVING_YDS' in row and pd.notna(row.get('RECEIVING_YDS')):
        points += row['RECEIVING_YDS'] * scoring_config['receiving']['yards']
    if 'RECEIVING_TDS' in row and pd.notna(row.get('RECEIVING_TDS')):
        points += row['RECEIVING_TDS'] * scoring_config['receiving']['touchdown']
    
    # Fumbles
    if 'MISC_FL' in row and pd.notna(row.get('MISC_FL')):
        points += row['MISC_FL'] * scoring_config['miscellaneous']['fumble_lost']
    
    return points

# Apply scoring
if projections is not None:
    projections['FANTASY_PTS'] = projections.apply(
        lambda row: calculate_fantasy_points(row, scoring), axis=1
    )
    
    # Sort by fantasy points
    projections = projections.sort_values('FANTASY_PTS', ascending=False)
    
    print("Top 20 Players by Projected Fantasy Points:")
    display(projections[['UNNAMED:_0_LEVEL_0_PLAYER', 'POSITION', 'MISC_FPTS', 'FANTASY_PTS']].head(20))

Top 20 Players by Projected Fantasy Points:


,UNNAMED:_0_LEVEL_0_PLAYER,POSITION,MISC_FPTS,FANTASY_PTS
1,Lamar Jackson BAL,QB,375.6,367.518
0,Josh Allen BUF,QB,376.7,366.168
2,Jalen Hurts PHI,QB,358.7,349.572
3,Jayden Daniels WAS,QB,355.8,345.452
4,Joe Burrow CIN,QB,341.6,331.046
5,Patrick Mahomes II KC,QB,328.8,317.026
7,Kyler Murray ARI,QB,319.1,307.986
6,Baker Mayfield TB,QB,321.6,307.236
8,Brock Purdy SF,QB,312.1,299.124
9,Bo Nix DEN,QB,308.8,296.482


In [5]:
# Position-specific rankings
if projections is not None:
    for position in ['QB', 'RB', 'WR', 'TE']:
        pos_df = projections[projections['POSITION'] == position].copy()
        pos_df['POS_RANK'] = range(1, len(pos_df) + 1)
        
        print(f"\nTop 10 {position}s:")
        display(pos_df[['POS_RANK', 'UNNAMED:_0_LEVEL_0_PLAYER', 'MISC_FPTS', 'FANTASY_PTS']].head(10))


Top 10 QBs:


,POS_RANK,UNNAMED:_0_LEVEL_0_PLAYER,MISC_FPTS,FANTASY_PTS
1,1,Lamar Jackson BAL,375.6,367.518
0,2,Josh Allen BUF,376.7,366.168
2,3,Jalen Hurts PHI,358.7,349.572
3,4,Jayden Daniels WAS,355.8,345.452
4,5,Joe Burrow CIN,341.6,331.046
5,6,Patrick Mahomes II KC,328.8,317.026
7,7,Kyler Murray ARI,319.1,307.986
6,8,Baker Mayfield TB,321.6,307.236
8,9,Brock Purdy SF,312.1,299.124
9,10,Bo Nix DEN,308.8,296.482



Top 10 RBs:


,POS_RANK,UNNAMED:_0_LEVEL_0_PLAYER,MISC_FPTS,FANTASY_PTS
102,1,Saquon Barkley PHI,273.1,273.17
103,2,Bijan Robinson ATL,263.2,262.99
104,3,Jahmyr Gibbs DET,258.9,259.23
105,4,Derrick Henry BAL,248.4,248.56
106,5,Christian McCaffrey SF,228.4,228.67
107,6,Josh Jacobs GB,226.1,226.58
108,7,De'Von Achane MIA,223.1,223.46
109,8,Ashton Jeanty LV,217.4,217.63
110,9,Jonathan Taylor IND,216.5,216.36
111,10,Kyren Williams LAR,213.0,213.42



Top 10 WRs:


,POS_RANK,UNNAMED:_0_LEVEL_0_PLAYER,MISC_FPTS,FANTASY_PTS
290,1,Ja'Marr Chase CIN,231.2,231.04
291,2,Justin Jefferson MIN,205.1,205.11
292,3,CeeDee Lamb DAL,193.3,193.18
293,4,Puka Nacua LAR,187.7,187.96
294,5,Nico Collins HOU,183.5,183.90
295,6,Malik Nabers NYG,183.3,183.07
296,7,Brian Thomas Jr. JAC,182.1,182.18
297,8,Amon-Ra St. Brown DET,180.9,181.43
298,9,A.J. Brown PHI,172.8,173.07
299,10,Drake London ATL,169.8,169.97



Top 10 TEs:


,POS_RANK,UNNAMED:_0_LEVEL_0_PLAYER,MISC_FPTS,FANTASY_PTS
521,1,Brock Bowers LV,149.7,148.75
522,2,George Kittle SF,147.6,147.71
523,3,Trey McBride ARI,139.4,139.13
524,4,Mark Andrews BAL,117.3,117.30
525,5,Sam LaPorta DET,115.6,115.79
526,6,Travis Kelce KC,106.9,106.90
527,7,T.J. Hockenson MIN,106.6,106.52
528,8,David Njoku CLE,100.4,100.35
529,9,Tucker Kraft GB,92.5,92.49
530,10,Tyler Warren IND,88.6,88.35


In [6]:
# Save processed rankings
if projections is not None:
    output_file = f'data/rankings_{datetime.now().strftime("%Y%m%d")}.csv'
    projections.to_csv(output_file, index=False)
    print(f"Rankings saved to: {output_file}")

Rankings saved to: data/rankings_20250814.csv
